In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('epi_r.csv')
join_df = pd.read_json('train.json')

In [3]:
df.dropna(inplace=True)
join_df.dropna(inplace=True)
df.set_index('title', inplace=True)
join_df.set_index('title', inplace=True)
df = df.loc[join_df.index]

C:\Users\marc.ribalta\AppData\Local\Continuum\anaconda3\envs\SIGNN\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """


In [4]:
df.index

Index(['Boudin Blanc Terrine with Red Onion Confit ', 'The Best Blts ',
       'Korean Marinated Beef ',
       'Ham Persillade with Mustard Potato Salad and Mashed Peas ',
       'Yams Braised with Cream, Rosemary and Nutmeg ',
       'Banana-Chocolate Chip Cake With Peanut Butter Frosting ',
       'Beef Tenderloin with Garlic and Brandy ', 'Peach Mustard ',
       'Peach Mustard ', 'Raw Cream of Spinach Soup ',
       ...
       'Chicken with White Wine and Herbs ',
       'Crispy Salt and Pepper Chicken with Caramelized Fennel and Shallots ',
       'Blueberry Compote ', 'Blueberry Compote ', 'Blueberry Compote ',
       'Blueberry Compote ', 'Blueberry Compote ',
       'Artichoke and Parmesan Risotto ',
       'Snapper on Angel Hair with Citrus Cream ',
       'Baked Ham with Marmalade-Horseradish Glaze '],
      dtype='object', name='title', length=14562)

In [5]:
df.head()

,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,30 days of groceries,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
title,,,,,,,,,,,,,,,,,,,,,
Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Best Blts,4.375,948.0,19.0,79.0,1042.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Korean Marinated Beef,4.375,170.0,7.0,10.0,1272.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ham Persillade with Mustard Potato Salad and Mashed Peas,3.750,602.0,23.0,41.0,1696.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Yams Braised with Cream, Rosemary and Nutmeg",3.750,256.0,4.0,5.0,30.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import random
ITERATIONS = 10 # Numero de cops que es fa la iteracio
columns = df.columns[11:].tolist()
columns.append('ratings')
final_df = pd.DataFrame(columns = columns)
df.dropna(inplace=True)
for ind in range(df.index.size):
    row = df.iloc[ind]
    if row.rating > 2.5:
        ma = 5
        mi = row.rating - (5-row.rating)
    else:
        ma = row.rating * 2
        mi = 0
    if (ma-mi)==0:
        ratings = [5] * 10 if ma==5 else [0] * 10
    else:
        ratings = np.arange(mi, ma, (ma-mi)/ITERATIONS)
    ingredients = row[11:]
    for i in range(1, ITERATIONS+1):
        available = ingredients[ingredients == 1]
        # Calculate how many ing are we going to get
        # Get ing from available
        # Create vector
        num_ing = int((len(available) * i)/ITERATIONS)
        num_ing = 1 if num_ing==0 and len(available)!=0 else num_ing
        picks = random.sample(range(len(available)), num_ing)
        ingr = [available.index.values[x] for x in picks]
        vector = (ingredients.index.values[11:], ingr, ratings[i-1])
        temp = pd.DataFrame(columns = columns)
        temp[df.columns[11:]] = 0.0
        temp[ingr] = 1.0
        temp['ratings'] = ratings[i-1]
        final_df.append(temp)
final_df

# Preparem per predir

Si tenim un usuari que li agraden els ingredients 1..n, li agradara aquesta recepta?

Per formular això, tenim un historic d'usuaris valoran una recepta, tenint els ingredients que li agraden al usuari en especific. Per tant tindrem nom de recepta, rating que li ha donat l'usuari, ingredients que li agraden al usuari i finalment la variable objectiu serà si li agradà a l'usuari o no. 

Per fer el train, assumirem que si un usuari ha fet un rating de més de 4, vol dir que li agrada la recepta,

Per crear opinions dels usuaris, tindrem que els millors casos son rating 5 i que tots els ingredients de la recepta li agraden al usuari. Per tant podem quadrar el numero de ingredients que li agraden a una persona tenint en compte els ingredients necessaris per fer una recepta. Mirarem primer de tot els ingredients necessaris per fer una recepta i crearem usuaris amb ingredients que agraden, de 100% i anar reduint fins a 0%. Recordem que s'ha d'afegir un factor random, i que algunes de les opinions tindran un factor random, per aixi perdre correlació.

In [ ]:
np.arange(4.375, 5, (5-4.375)/10)

In [ ]:
a = df.iloc[0][11:]
a[a == 1]